# ĐỀ TÀI NCKH: KHAI PHÁ LUẬT KẾT HỢP TRONG DỮ LIỆU BÁN LẺ
### (Market Basket Analysis using Apriori & FP-Growth Algorithms)

---
**Thông tin sinh viên thực hiện:**
* **Họ và tên:** Huỳnh Phúc Hưng
* **MSSV:** 3122411073
* **Lớp:** DCT122C3
* **Trường:** Đại học Sài Gòn
---

## 1. Giới thiệu tổng quan
Trong ngành bán lẻ hiện đại, việc hiểu rõ hành vi mua sắm của khách hàng là chìa khóa để tối ưu hóa doanh thu. **Phân tích giỏ hàng (Market Basket Analysis)** là kỹ thuật khai phá dữ liệu giúp trả lời câu hỏi: *"Khách hàng mua sản phẩm A thường sẽ mua kèm sản phẩm nào?"*.

**Mục tiêu của Notebook này:**
1.  Sử dụng bộ dữ liệu **Groceries Dataset** (từ Kaggle) chứa các giao dịch thực tế.
2.  Áp dụng hai thuật toán phổ biến nhất:
    * **Apriori:** Thuật toán kinh điển trong khai phá luật kết hợp.
    * **FP-Growth (Frequent Pattern Growth):** Thuật toán cải tiến với tốc độ xử lý nhanh hơn.
3.  So sánh hiệu năng giữa hai thuật toán và đưa ra các **Luật kết hợp (Association Rules)** có giá trị thực tiễn (ví dụ: Sắp xếp quầy kệ, gợi ý combo khuyến mãi).

## 2. Mô tả dữ liệu
* **Nguồn dữ liệu:** Groceries Dataset (Kaggle).
* **Đặc điểm:** Dữ liệu chứa thông tin giao dịch của khách hàng, bao gồm mã khách hàng, ngày mua và tên sản phẩm.
* **Công cụ thực hiện:** Python (thư viện `mlxtend`, `pandas`, `seaborn`).

---
*(Bắt đầu quy trình xử lý dữ liệu bên dưới)*

### GIAI ĐOẠN 1: TỔNG QUAN VÀ CHUẨN BỊ

In [ ]:
# Cài đặt thư viện mlxtend
!pip install mlxtend --upgrade

In [2]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

print("Đã cài đặt và import xong các thư viện!")

Đã cài đặt và import xong các thư viện!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### Trước khi bước vào làm, chúng ta phải làm rõ 3 khái niệm sau: Độ hỗ trợ, độ tin cậy, độ nâng.

Giả sử bạn có 100 hóa đơn siêu thị.

1. Độ hỗ trợ (Support): Độ phổ biến của món hàng.

Ví dụ: Có 10 hóa đơn mua Sữa. => Support(Sữa) = 10/100 = 10%.

Ý nghĩa: Loại bỏ các món quá ít người mua để đỡ tốn công tính toán.

2. Độ tin cậy (Confidence): Xác suất mua B khi đã mua A.

Ví dụ: Trong 10 người mua Sữa, có 7 người mua thêm Bánh mì. => Confidence(Sữa -> Bánh mì) = 7/10 = 70%.

Ý nghĩa: Thể hiện độ mạnh của luật kết hợp.

3. Độ nâng (Lift): (Quan trọng nhất) Đo lường sự ngẫu nhiên.

Lift = 1: Mua A và B chẳng liên quan gì nhau (ngẫu nhiên).

Lift > 1: Mua A thúc đẩy việc mua B (Đây là cái ta cần tìm).

Lift < 1: Mua A thì người ta né không mua B.

### GIAI ĐOẠN 2: TIỀN XỬ LÝ DỮ LIỆU

In [3]:
import pandas as pd

# 1. Đọc dữ liệu từ file CSV
df = pd.read_csv('Groceries_dataset.csv')

# 2. Xem thử 5 dòng đầu tiên để hiểu dữ liệu
print("--- Dữ liệu gốc ---")
print(df.head())

# 3. Gom nhóm: Một người (Member_number) đi chợ vào một ngày (Date) tính là 1 giỏ hàng
# Chúng ta sẽ gom tất cả món hàng (itemDescription) của cùng 1 người trong 1 ngày lại thành 1 danh sách
transactions_df = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index(name='items')

print("\n--- Dữ liệu sau khi xử lý (Dạng giỏ hàng) ---")
print(transactions_df.head())

# 4. Kiểm tra số lượng
print(f"\nTổng số giao dịch (giỏ hàng) tìm được: {len(transactions_df)}")

# 5. Lưu danh sách giỏ hàng vào biến 'dataset' để dùng cho thuật toán
dataset = transactions_df['items'].tolist()
print(f"Ví dụ giỏ hàng đầu tiên: {dataset[0]}")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

--- Dữ liệu gốc ---
   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- Dữ liệu sau khi xử lý (Dạng giỏ hàng) ---
   Member_number        Date  \
0           1000  15-03-2015   
1           1000  24-06-2014   
2           1000  24-07-2015   
3           1000  25-11-2015   
4           1000  27-05-2015   

                                               items  
0  [sausage, whole milk, semi-finished bread, yog...  
1                  [whole milk, pastry, salty snack]  
2                     [canned beer, misc. beverages]  
3                        [sausage, hygiene articles]  
4                         [soda, pickled vegetables]  

Tổng số giao dịch (giỏ hàng) tìm được: 14963
Ví dụ giỏ hàng đầu tiên: ['sausage', 'whole milk', 'semi-finished bread', 'yogurt']


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag